In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Train (2).csv')
test_given = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Test (2).csv')
sample_submission = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/SampleSubmission (2).csv')

# Encoding categorical variables
train['form_field47'].replace('charge',1, inplace=True)
train['form_field47'].replace('lending',0, inplace=True)
train['default_status'].replace('yes',1, inplace=True)
train['default_status'].replace('no',0, inplace=True)

test_given['form_field47'].replace('charge',1, inplace=True)
test_given['form_field47'].replace('lending',0, inplace=True)


new_features = train.select_dtypes(exclude = object).columns.drop(['default_status','form_field23','form_field45','form_field11', 'form_field4','form_field17','form_field18','form_field30','form_field31','form_field35','form_field39'])
test_new_features = test_given.select_dtypes(exclude = object).columns.drop(['form_field23','form_field45','form_field11','form_field4','form_field17','form_field18','form_field30','form_field31','form_field35','form_field39'])

for i in new_features:
    train[i].fillna(-999, inplace=True)
    test_given[i].fillna(-999, inplace=True)

test_given_new = test_given[test_new_features]
test_given_new.head()

#train[new_features].isna().sum()

X = train[new_features]
Y = train.iloc[:,-1]

from sklearn.model_selection import StratifiedKFold

n_splits = 15
params = {'n_estimators':4000, 'learning_rate':0.01, 'objective':'CrossEntropy','eval_metric':'AUC',
          'random_seed': 42, 'max_depth':7,'early_stopping_rounds':200,'use_best_model':True}
fold=StratifiedKFold(n_splits)

from sklearn.metrics import roc_auc_score

def metric(Y,pred):
    return roc_auc_score(Y,pred, labels=[0,1])

from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

list_scores = []
test_oofs = []
score = 0 

for i, (tr_idx, vr_idx) in enumerate(fold.split(X,Y)):
    X_train, Y_train = X.loc[tr_idx, new_features], Y.loc[tr_idx]
    X_val, Y_val = X.loc[vr_idx,new_features], Y.loc[vr_idx]
    
    model= CatBoostClassifier(**params)
    model.fit(X_train,Y_train,eval_set=[(X_val,Y_val)],verbose=100,)#erly100
    
    pred = model.predict_proba(X_val)[:,1]
    sc = metric(Y_val, pred)
    list_scores.append(sc)
    score += sc/n_splits
    
    pred_test = model.predict_proba(test_given_new[test_new_features])[:,1]
    test_oofs.append(pred_test)

0:	test: 0.7870677	best: 0.7870677 (0)	total: 180ms	remaining: 11m 59s
100:	test: 0.8165658	best: 0.8165658 (100)	total: 6.89s	remaining: 4m 26s
200:	test: 0.8205853	best: 0.8205853 (200)	total: 13.3s	remaining: 4m 12s
300:	test: 0.8230028	best: 0.8230028 (300)	total: 19.3s	remaining: 3m 56s
400:	test: 0.8245338	best: 0.8245520 (398)	total: 25.3s	remaining: 3m 47s
500:	test: 0.8258131	best: 0.8258131 (500)	total: 31.4s	remaining: 3m 39s
600:	test: 0.8267967	best: 0.8267967 (600)	total: 37.7s	remaining: 3m 33s
700:	test: 0.8277175	best: 0.8277291 (697)	total: 43.7s	remaining: 3m 25s
800:	test: 0.8281451	best: 0.8281559 (794)	total: 49.7s	remaining: 3m 18s
900:	test: 0.8285494	best: 0.8285646 (883)	total: 55.7s	remaining: 3m 11s
1000:	test: 0.8289623	best: 0.8289705 (999)	total: 1m 1s	remaining: 3m 4s
1100:	test: 0.8294590	best: 0.8294675 (1087)	total: 1m 7s	remaining: 2m 57s
1200:	test: 0.8298874	best: 0.8298924 (1193)	total: 1m 13s	remaining: 2m 51s
1300:	test: 0.8300975	best: 0.830097

1700:	test: 0.8235649	best: 0.8235785 (1697)	total: 1m 54s	remaining: 2m 34s
1800:	test: 0.8236963	best: 0.8237386 (1789)	total: 2m 1s	remaining: 2m 27s
1900:	test: 0.8238549	best: 0.8238696 (1897)	total: 2m 7s	remaining: 2m 20s
2000:	test: 0.8239530	best: 0.8239553 (1999)	total: 2m 14s	remaining: 2m 14s
2100:	test: 0.8239809	best: 0.8239929 (2063)	total: 2m 20s	remaining: 2m 7s
2200:	test: 0.8239262	best: 0.8239929 (2063)	total: 2m 26s	remaining: 2m
2300:	test: 0.8239929	best: 0.8240243 (2226)	total: 2m 33s	remaining: 1m 53s
2400:	test: 0.8239387	best: 0.8240329 (2328)	total: 2m 39s	remaining: 1m 46s
2500:	test: 0.8239224	best: 0.8240329 (2328)	total: 2m 46s	remaining: 1m 39s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8240328605
bestIteration = 2328

Shrink model to first 2329 iterations.
0:	test: 0.7874188	best: 0.7874188 (0)	total: 122ms	remaining: 8m 9s
100:	test: 0.8262540	best: 0.8262540 (100)	total: 7.18s	remaining: 4m 37s
200:	test: 0.8323588	best: 0.8

2700:	test: 0.8427061	best: 0.8429076 (2527)	total: 2m 56s	remaining: 1m 24s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8429075754
bestIteration = 2527

Shrink model to first 2528 iterations.
0:	test: 0.7717415	best: 0.7717415 (0)	total: 136ms	remaining: 9m 4s
100:	test: 0.8109998	best: 0.8109998 (100)	total: 7.73s	remaining: 4m 58s
200:	test: 0.8134195	best: 0.8134195 (200)	total: 14.3s	remaining: 4m 29s
300:	test: 0.8152223	best: 0.8152223 (300)	total: 20.7s	remaining: 4m 14s
400:	test: 0.8163292	best: 0.8163292 (400)	total: 27.2s	remaining: 4m 3s
500:	test: 0.8174485	best: 0.8174485 (500)	total: 33.7s	remaining: 3m 55s
600:	test: 0.8181448	best: 0.8181448 (600)	total: 40.3s	remaining: 3m 48s
700:	test: 0.8186388	best: 0.8186388 (700)	total: 46.9s	remaining: 3m 40s
800:	test: 0.8191376	best: 0.8191376 (800)	total: 53.5s	remaining: 3m 33s
900:	test: 0.8196134	best: 0.8196134 (900)	total: 60s	remaining: 3m 26s
1000:	test: 0.8199596	best: 0.8199658 (999)	total:

1100:	test: 0.8479585	best: 0.8479585 (1100)	total: 1m 12s	remaining: 3m 11s
1200:	test: 0.8484463	best: 0.8484626 (1199)	total: 1m 19s	remaining: 3m 4s
1300:	test: 0.8487875	best: 0.8487875 (1300)	total: 1m 25s	remaining: 2m 57s
1400:	test: 0.8491023	best: 0.8491065 (1390)	total: 1m 32s	remaining: 2m 50s
1500:	test: 0.8494593	best: 0.8494593 (1500)	total: 1m 38s	remaining: 2m 44s
1600:	test: 0.8496281	best: 0.8496619 (1595)	total: 1m 45s	remaining: 2m 37s
1700:	test: 0.8497163	best: 0.8497869 (1637)	total: 1m 51s	remaining: 2m 30s
1800:	test: 0.8498439	best: 0.8498439 (1800)	total: 1m 58s	remaining: 2m 24s
1900:	test: 0.8498404	best: 0.8499014 (1892)	total: 2m 4s	remaining: 2m 17s
2000:	test: 0.8498183	best: 0.8499014 (1892)	total: 2m 11s	remaining: 2m 11s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8499013804
bestIteration = 1892

Shrink model to first 1893 iterations.


In [3]:
oof_pred = pd.DataFrame(test_oofs).T
oof_pred.columns = ['split_' + str(i) for i in range(1, n_splits+1)]
oof_pred.head()

sample_submission['default_status'] = np.mean(test_oofs, axis = 0)

sample_submission.to_csv('Submission_new1.csv', index=False)

In [5]:
#Best
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Train (2).csv')
test_given = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Test (2).csv')
sample_submission = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/SampleSubmission (2).csv')

# Encoding categorical variables
train['form_field47'].replace('charge',1, inplace=True)
train['form_field47'].replace('lending',0, inplace=True)
train['default_status'].replace('yes',1, inplace=True)
train['default_status'].replace('no',0, inplace=True)

test_given['form_field47'].replace('charge',1, inplace=True)
test_given['form_field47'].replace('lending',0, inplace=True)


new_features = train.select_dtypes(exclude = object).columns.drop(['default_status','form_field23','form_field45','form_field11', 'form_field4','form_field17','form_field18','form_field30','form_field31','form_field35','form_field39'])
test_new_features = test_given.select_dtypes(exclude = object).columns.drop(['form_field23','form_field45','form_field11','form_field4','form_field17','form_field18','form_field30','form_field31','form_field35','form_field39'])

for i in new_features:
    train[i].fillna(-999, inplace=True)
    test_given[i].fillna(-999, inplace=True)

test_given_new = test_given[test_new_features]
test_given_new.head()

#train[new_features].isna().sum()

X = train[new_features]
Y = train.iloc[:,-1]

from sklearn.model_selection import StratifiedKFold

n_splits = 15
params = {'n_estimators':4000, 'learning_rate':0.01, 'objective':'CrossEntropy','eval_metric':'AUC',
          'random_seed': 42, 'max_depth':8,'early_stopping_rounds':200,'use_best_model':True}
fold=StratifiedKFold(n_splits)

from sklearn.metrics import roc_auc_score

def metric(Y,pred):
    return roc_auc_score(Y,pred, labels=[0,1])

from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

list_scores = []
test_oofs = []
score = 0 

for i, (tr_idx, vr_idx) in enumerate(fold.split(X,Y)):
    X_train, Y_train = X.loc[tr_idx, new_features], Y.loc[tr_idx]
    X_val, Y_val = X.loc[vr_idx,new_features], Y.loc[vr_idx]
    
    model= CatBoostClassifier(**params)
    model.fit(X_train,Y_train,eval_set=[(X_val,Y_val)],verbose=100,)#erly100
    
    pred = model.predict_proba(X_val)[:,1]
    sc = metric(Y_val, pred)
    list_scores.append(sc)
    score += sc/n_splits
    
    pred_test = model.predict_proba(test_given_new[test_new_features])[:,1]
    test_oofs.append(pred_test)

0:	test: 0.7862938	best: 0.7862938 (0)	total: 169ms	remaining: 11m 16s
100:	test: 0.8180153	best: 0.8180153 (100)	total: 9.37s	remaining: 6m 1s
200:	test: 0.8215183	best: 0.8215183 (200)	total: 17.7s	remaining: 5m 35s
300:	test: 0.8240239	best: 0.8240239 (300)	total: 26s	remaining: 5m 19s
400:	test: 0.8255480	best: 0.8255677 (396)	total: 34.2s	remaining: 5m 7s
500:	test: 0.8270747	best: 0.8270747 (500)	total: 42.4s	remaining: 4m 56s
600:	test: 0.8279586	best: 0.8279718 (599)	total: 50.7s	remaining: 4m 46s
700:	test: 0.8286064	best: 0.8286115 (698)	total: 58.9s	remaining: 4m 37s
800:	test: 0.8289689	best: 0.8289689 (800)	total: 1m 7s	remaining: 4m 28s
900:	test: 0.8295152	best: 0.8295256 (897)	total: 1m 15s	remaining: 4m 19s
1000:	test: 0.8298238	best: 0.8298463 (996)	total: 1m 23s	remaining: 4m 10s
1100:	test: 0.8301083	best: 0.8301281 (1096)	total: 1m 31s	remaining: 4m 1s
1200:	test: 0.8302293	best: 0.8302390 (1191)	total: 1m 40s	remaining: 3m 53s
1300:	test: 0.8304673	best: 0.8304673

1200:	test: 0.8223937	best: 0.8224317 (1193)	total: 1m 46s	remaining: 4m 7s
1300:	test: 0.8225213	best: 0.8225321 (1253)	total: 1m 54s	remaining: 3m 58s
1400:	test: 0.8227589	best: 0.8227837 (1394)	total: 2m 3s	remaining: 3m 49s
1500:	test: 0.8227721	best: 0.8228872 (1447)	total: 2m 12s	remaining: 3m 41s
1600:	test: 0.8229206	best: 0.8229590 (1570)	total: 2m 22s	remaining: 3m 32s
1700:	test: 0.8230218	best: 0.8230846 (1694)	total: 2m 30s	remaining: 3m 23s
1800:	test: 0.8229780	best: 0.8230974 (1724)	total: 2m 39s	remaining: 3m 14s
1900:	test: 0.8230101	best: 0.8230974 (1724)	total: 2m 48s	remaining: 3m 5s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8230973662
bestIteration = 1724

Shrink model to first 1725 iterations.
0:	test: 0.8024890	best: 0.8024890 (0)	total: 166ms	remaining: 11m 5s
100:	test: 0.8282829	best: 0.8282829 (100)	total: 9.72s	remaining: 6m 15s
200:	test: 0.8333511	best: 0.8333511 (200)	total: 18.5s	remaining: 5m 50s
300:	test: 0.8358424	best: 0

1600:	test: 0.8219541	best: 0.8220263 (1470)	total: 2m 23s	remaining: 3m 34s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8220262939
bestIteration = 1470

Shrink model to first 1471 iterations.
0:	test: 0.7707771	best: 0.7707771 (0)	total: 182ms	remaining: 12m 8s
100:	test: 0.8026767	best: 0.8026767 (100)	total: 10.1s	remaining: 6m 30s
200:	test: 0.8094099	best: 0.8094099 (200)	total: 18.9s	remaining: 5m 57s
300:	test: 0.8128571	best: 0.8128571 (300)	total: 27.6s	remaining: 5m 39s
400:	test: 0.8148093	best: 0.8148163 (399)	total: 36.4s	remaining: 5m 26s
500:	test: 0.8160738	best: 0.8160800 (496)	total: 45.2s	remaining: 5m 15s
600:	test: 0.8167872	best: 0.8168326 (596)	total: 54s	remaining: 5m 5s
700:	test: 0.8171225	best: 0.8171225 (700)	total: 1m 2s	remaining: 4m 55s
800:	test: 0.8174140	best: 0.8174287 (793)	total: 1m 11s	remaining: 4m 46s
900:	test: 0.8176883	best: 0.8176883 (900)	total: 1m 20s	remaining: 4m 37s
1000:	test: 0.8178203	best: 0.8178292 (987)	tot

In [6]:
oof_pred1 = pd.DataFrame(test_oofs).T
oof_pred1.columns = ['split_' + str(i) for i in range(1, n_splits+1)]
oof_pred1.head()

sample_submission['default_status'] = np.mean(test_oofs, axis = 0)

sample_submission.to_csv('Submission_new2.csv', index=False)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Train (2).csv')
test_given = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Test (2).csv')
sample_submission = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/SampleSubmission (2).csv')

# Encoding categorical variables
train['form_field47'].replace('charge',1, inplace=True)
train['form_field47'].replace('lending',0, inplace=True)
train['default_status'].replace('yes',1, inplace=True)
train['default_status'].replace('no',0, inplace=True)

test_given['form_field47'].replace('charge',1, inplace=True)
test_given['form_field47'].replace('lending',0, inplace=True)


new_features = train.select_dtypes(exclude = object).columns.drop(['default_status','form_field23','form_field45','form_field11', 'form_field4','form_field17','form_field18','form_field30','form_field31','form_field35','form_field39'])
test_new_features = test_given.select_dtypes(exclude = object).columns.drop(['form_field23','form_field45','form_field11','form_field4','form_field17','form_field18','form_field30','form_field31','form_field35','form_field39'])

for i in new_features:
    train[i].fillna(-999, inplace=True)
    test_given[i].fillna(-999, inplace=True)

test_given_new = test_given[test_new_features]
test_given_new.head()

#train[new_features].isna().sum()

X = train[new_features]
Y = train.iloc[:,-1]

from sklearn.model_selection import StratifiedKFold

n_splits = 15
params = {'n_estimators':4000, 'learning_rate':0.01, 'objective':'CrossEntropy','eval_metric':'AUC',
          'random_seed': 42, 'max_depth':9,'early_stopping_rounds':200,'use_best_model':True}
fold=StratifiedKFold(n_splits)

from sklearn.metrics import roc_auc_score

def metric(Y,pred):
    return roc_auc_score(Y,pred, labels=[0,1])

from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

list_scores = []
test_oofs = []
score = 0 

for i, (tr_idx, vr_idx) in enumerate(fold.split(X,Y)):
    X_train, Y_train = X.loc[tr_idx, new_features], Y.loc[tr_idx]
    X_val, Y_val = X.loc[vr_idx,new_features], Y.loc[vr_idx]
    
    model= CatBoostClassifier(**params)
    model.fit(X_train,Y_train,eval_set=[(X_val,Y_val)],verbose=100,)#erly100
    
    pred = model.predict_proba(X_val)[:,1]
    sc = metric(Y_val, pred)
    list_scores.append(sc)
    score += sc/n_splits
    
    pred_test = model.predict_proba(test_given_new[test_new_features])[:,1]
    test_oofs.append(pred_test)

0:	test: 0.7913192	best: 0.7913192 (0)	total: 356ms	remaining: 23m 42s
100:	test: 0.8197853	best: 0.8197853 (100)	total: 15.7s	remaining: 10m 4s
200:	test: 0.8233284	best: 0.8233311 (199)	total: 29.7s	remaining: 9m 21s
300:	test: 0.8251870	best: 0.8251870 (300)	total: 45.7s	remaining: 9m 21s
400:	test: 0.8267634	best: 0.8267634 (400)	total: 1m	remaining: 9m 4s
500:	test: 0.8278194	best: 0.8278194 (500)	total: 1m 15s	remaining: 8m 45s
600:	test: 0.8285925	best: 0.8286258 (594)	total: 1m 29s	remaining: 8m 27s
700:	test: 0.8293566	best: 0.8293582 (698)	total: 1m 45s	remaining: 8m 17s
800:	test: 0.8298420	best: 0.8298668 (797)	total: 2m	remaining: 8m
900:	test: 0.8301991	best: 0.8302188 (892)	total: 2m 14s	remaining: 7m 43s
1000:	test: 0.8303910	best: 0.8304433 (988)	total: 2m 29s	remaining: 7m 27s
1100:	test: 0.8306069	best: 0.8306069 (1100)	total: 2m 43s	remaining: 7m 11s
1200:	test: 0.8310074	best: 0.8310144 (1199)	total: 2m 58s	remaining: 6m 56s
1300:	test: 0.8313935	best: 0.8313935 (1

100:	test: 0.8296706	best: 0.8296706 (100)	total: 17.6s	remaining: 11m 20s
200:	test: 0.8339231	best: 0.8339231 (200)	total: 34.6s	remaining: 10m 54s
300:	test: 0.8360395	best: 0.8360636 (298)	total: 51.4s	remaining: 10m 32s
400:	test: 0.8373913	best: 0.8373913 (400)	total: 1m 8s	remaining: 10m 10s
500:	test: 0.8385165	best: 0.8385165 (500)	total: 1m 24s	remaining: 9m 51s
600:	test: 0.8393265	best: 0.8393265 (600)	total: 1m 41s	remaining: 9m 32s
700:	test: 0.8398302	best: 0.8398376 (698)	total: 1m 58s	remaining: 9m 15s
800:	test: 0.8401974	best: 0.8402094 (797)	total: 2m 14s	remaining: 8m 57s
900:	test: 0.8404652	best: 0.8405094 (879)	total: 2m 31s	remaining: 8m 39s
1000:	test: 0.8411929	best: 0.8411929 (1000)	total: 2m 47s	remaining: 8m 22s
1100:	test: 0.8413683	best: 0.8413889 (1080)	total: 3m 4s	remaining: 8m 5s
1200:	test: 0.8416369	best: 0.8416633 (1179)	total: 3m 20s	remaining: 7m 48s
1300:	test: 0.8420592	best: 0.8420592 (1300)	total: 3m 37s	remaining: 7m 30s
1400:	test: 0.84241

1100:	test: 0.8177275	best: 0.8177865 (1031)	total: 2m 51s	remaining: 7m 30s
1200:	test: 0.8177229	best: 0.8177896 (1144)	total: 3m 6s	remaining: 7m 15s
1300:	test: 0.8178832	best: 0.8178832 (1300)	total: 3m 22s	remaining: 6m 59s
1400:	test: 0.8180749	best: 0.8181230 (1385)	total: 3m 37s	remaining: 6m 44s
1500:	test: 0.8179468	best: 0.8181230 (1385)	total: 3m 53s	remaining: 6m 28s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8181230366
bestIteration = 1385

Shrink model to first 1386 iterations.
0:	test: 0.8056564	best: 0.8056564 (0)	total: 229ms	remaining: 15m 17s
100:	test: 0.8384551	best: 0.8384551 (100)	total: 16.3s	remaining: 10m 27s
200:	test: 0.8428692	best: 0.8428692 (200)	total: 31.7s	remaining: 9m 58s
300:	test: 0.8441736	best: 0.8441736 (300)	total: 46.8s	remaining: 9m 35s
400:	test: 0.8452324	best: 0.8452401 (398)	total: 1m 1s	remaining: 9m 14s
500:	test: 0.8463870	best: 0.8463870 (500)	total: 1m 17s	remaining: 8m 58s
600:	test: 0.8469863	best: 0.846

In [2]:
oof_pred1 = pd.DataFrame(test_oofs).T
oof_pred1.columns = ['split_' + str(i) for i in range(1, n_splits+1)]
oof_pred1.head()

sample_submission['default_status'] = np.mean(test_oofs, axis = 0)

sample_submission.to_csv('Submission_new3.csv', index=False)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Train (2).csv')
test_given = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/Test (2).csv')
sample_submission = pd.read_csv('C:/Users/USER/Desktop/DSN Hackathon/Bootcamp Qualification/SampleSubmission (2).csv')

# Encoding categorical variables
train['form_field47'].replace('charge',1, inplace=True)
train['form_field47'].replace('lending',0, inplace=True)
train['default_status'].replace('yes',1, inplace=True)
train['default_status'].replace('no',0, inplace=True)

test_given['form_field47'].replace('charge',1, inplace=True)
test_given['form_field47'].replace('lending',0, inplace=True)


new_features = train.select_dtypes(exclude = object).columns.drop(['default_status','form_field23','form_field45','form_field11', 'form_field4','form_field17','form_field18','form_field30','form_field31','form_field35','form_field39'])
test_new_features = test_given.select_dtypes(exclude = object).columns.drop(['form_field23','form_field45','form_field11','form_field4','form_field17','form_field18','form_field30','form_field31','form_field35','form_field39'])

for i in new_features:
    train[i].fillna(-999, inplace=True)
    test_given[i].fillna(-999, inplace=True)

test_given_new = test_given[test_new_features]
test_given_new.head()

#train[new_features].isna().sum()

X = train[new_features]
Y = train.iloc[:,-1]

from sklearn.model_selection import StratifiedKFold

n_splits = 15
params = {'n_estimators':4000, 'learning_rate':0.01, 'objective':'CrossEntropy','eval_metric':'AUC',
          'random_seed': 42, 'max_depth':9,'early_stopping_rounds':200,'use_best_model':True}
fold=StratifiedKFold(n_splits)

from sklearn.metrics import roc_auc_score

def metric(Y,pred):
    return roc_auc_score(Y,pred, labels=[0,1])

from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

list_scores = []
test_oofs = []
score = 0 

for i, (tr_idx, vr_idx) in enumerate(fold.split(X,Y)):
    X_train, Y_train = X.loc[tr_idx, new_features], Y.loc[tr_idx]
    X_val, Y_val = X.loc[vr_idx,new_features], Y.loc[vr_idx]
    
    model= CatBoostClassifier(**params)
    model.fit(X_train,Y_train,eval_set=[(X_val,Y_val)],verbose=100,)#erly100
    
    pred = model.predict_proba(X_val)[:,1]
    sc = metric(Y_val, pred)
    list_scores.append(sc)
    score += sc/n_splits
    
    pred_test = model.predict_proba(test_given_new[test_new_features])[:,1]
    test_oofs.append(pred_test)

0:	test: 0.7913192	best: 0.7913192 (0)	total: 1.1s	remaining: 1h 13m 42s
100:	test: 0.8197853	best: 0.8197853 (100)	total: 30.4s	remaining: 19m 32s
200:	test: 0.8233284	best: 0.8233311 (199)	total: 57.9s	remaining: 18m 15s
300:	test: 0.8251870	best: 0.8251870 (300)	total: 1m 25s	remaining: 17m 29s
400:	test: 0.8267634	best: 0.8267634 (400)	total: 1m 52s	remaining: 16m 46s
500:	test: 0.8278194	best: 0.8278194 (500)	total: 2m 19s	remaining: 16m 11s
600:	test: 0.8285925	best: 0.8286258 (594)	total: 2m 45s	remaining: 15m 38s
700:	test: 0.8293566	best: 0.8293582 (698)	total: 3m 12s	remaining: 15m 7s
800:	test: 0.8298420	best: 0.8298668 (797)	total: 3m 39s	remaining: 14m 37s
900:	test: 0.8301991	best: 0.8302188 (892)	total: 4m 7s	remaining: 14m 11s
1000:	test: 0.8303910	best: 0.8304433 (988)	total: 4m 35s	remaining: 13m 44s
1100:	test: 0.8306069	best: 0.8306069 (1100)	total: 5m 2s	remaining: 13m 17s
1200:	test: 0.8310074	best: 0.8310144 (1199)	total: 5m 29s	remaining: 12m 48s
1300:	test: 0.8

0:	test: 0.8017811	best: 0.8017811 (0)	total: 262ms	remaining: 17m 29s
100:	test: 0.8296706	best: 0.8296706 (100)	total: 28.6s	remaining: 18m 24s
200:	test: 0.8339231	best: 0.8339231 (200)	total: 57.6s	remaining: 18m 7s
300:	test: 0.8360395	best: 0.8360636 (298)	total: 1m 25s	remaining: 17m 35s
400:	test: 0.8373913	best: 0.8373913 (400)	total: 1m 53s	remaining: 17m
500:	test: 0.8385165	best: 0.8385165 (500)	total: 2m 21s	remaining: 16m 26s
600:	test: 0.8393265	best: 0.8393265 (600)	total: 2m 49s	remaining: 15m 58s
700:	test: 0.8398302	best: 0.8398376 (698)	total: 3m 17s	remaining: 15m 28s
800:	test: 0.8401974	best: 0.8402094 (797)	total: 3m 44s	remaining: 14m 58s
900:	test: 0.8404652	best: 0.8405094 (879)	total: 4m 12s	remaining: 14m 28s
1000:	test: 0.8411929	best: 0.8411929 (1000)	total: 4m 40s	remaining: 13m 59s
1100:	test: 0.8413683	best: 0.8413889 (1080)	total: 5m 7s	remaining: 13m 30s
1200:	test: 0.8416369	best: 0.8416633 (1179)	total: 5m 35s	remaining: 13m 1s
1300:	test: 0.842059

1000:	test: 0.8176165	best: 0.8176464 (996)	total: 2m 22s	remaining: 7m 7s
1100:	test: 0.8177275	best: 0.8177865 (1031)	total: 2m 36s	remaining: 6m 51s
1200:	test: 0.8177229	best: 0.8177896 (1144)	total: 2m 50s	remaining: 6m 36s
1300:	test: 0.8178832	best: 0.8178832 (1300)	total: 3m 3s	remaining: 6m 21s
1400:	test: 0.8180749	best: 0.8181230 (1385)	total: 3m 17s	remaining: 6m 6s
1500:	test: 0.8179468	best: 0.8181230 (1385)	total: 3m 31s	remaining: 5m 52s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8181230366
bestIteration = 1385

Shrink model to first 1386 iterations.
0:	test: 0.8056564	best: 0.8056564 (0)	total: 231ms	remaining: 15m 25s
100:	test: 0.8384551	best: 0.8384551 (100)	total: 14.7s	remaining: 9m 28s
200:	test: 0.8428692	best: 0.8428692 (200)	total: 28.5s	remaining: 8m 58s
300:	test: 0.8441736	best: 0.8441736 (300)	total: 42.4s	remaining: 8m 41s
400:	test: 0.8452324	best: 0.8452401 (398)	total: 56.2s	remaining: 8m 24s
500:	test: 0.8463870	best: 0.84638

In [2]:
oof_pred1 = pd.DataFrame(test_oofs).T
oof_pred1.columns = ['split_' + str(i) for i in range(1, n_splits+1)]
oof_pred1.head()

sample_submission['default_status'] = np.mean(test_oofs, axis = 0)

sample_submission.to_csv('Submision_new4.csv', index=False)